In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import re

In [ ]:
#import the data sets
cdc_data_path = "data/raw/CDC_health_outcomes/long_form"
cdc_file_list = os.listdir(cdc_data_path)

cdc_dict = {} #each year as a string will be key
for file in cdc_file_list:
    #get the release year
    release_year = re.search(r'([0-9]{4})_release', file).group(1)

    #dict containing data tyoes for loading
    type_map = {
        'Year': 'category',
        'StateAbbr': 'category',
        'Measure': 'category',
        "Data_Value_Unit":'category',
        "DataValueTypeID":'category',
        "Data_Value_Type":'category',
        "Data_Value": 'Float32',
        "Low_Confidence_Limit": 'Float32',
        "High_Confidence_Limit": 'Float32',
        "MeasureId":'category',
        'CityFIPS': 'str,',
        'TractFIPS': 'str'}
    converter_map ={
        "Population2010": lambda x: x.replace(",","")
    }
    #read in the data
    df = pd.read_csv(os.path.join(cdc_data_path, file), dtype=type_map, converters=converter_map )
    cdc_dict[release_year] = df


In [ ]:
cdc_dict['2016'].head()

In [ ]:
og_cols = set(cdc_dict['2016'])
for year, df in cdc_dict.items():
    new_cols = set(df)

    print("mising in", year, list(og_cols-new_cols))
    print("mising in 2016" , list(new_cols-og_cols))
    print()

In [ ]:
#Rename cols to match
cdc_dict['2018'] = cdc_dict['2018'].rename(columns= {'PopulationCount':'Population2010','Geolocation':'GeoLocation'})
cdc_dict['2019'] = cdc_dict['2019'].rename(columns= {'PopulationCount':'Population2010','Geolocation':'GeoLocation'})
#combine them together
cdc = pd.concat(cdc_dict).reset_index()


#rename the year column and drop old index
cdc = cdc.rename(columns={'level_0':'Release_Year', 'Year':'Survey_Year'})
del cdc['level_1']

In [ ]:
cdc.head()

In [ ]:
#there are duplicates becuase some surveys are only done every other year

cdc = cdc.drop_duplicates(subset=['Survey_Year', 'UniqueID', 'MeasureId', 'Data_Value_Type'])

In [ ]:
#I dont know if this is right, but going to delete all the city level data
cdc = cdc[cdc['GeographicLevel']=='Census Tract']

In [ ]:
#save 
cdc.to_csv('data/clean/cdc_500_cities.csv', index=False)

# Food Atlas

In [ ]:
#if starting here


In [ ]:
#import the data sets
file_list = ['data/raw/food_atlas/FoodAccessResearchAtlasData2015.xlsx',
             'data/raw/food_atlas/FoodAccessResearchAtlasData2019.xlsx']

atlas_dict = {} #each year as a string will be key
for file in file_list:
    #get the release year
    release_year = re.search(r'Data([0-9]{4})', file).group(1)

    #read in the data
    df = pd.read_excel(file, sheet_name = 2, dtype='str')
    atlas_dict[release_year] = df


In [79]:
#check all cols are same
og_cols = set(atlas_dict['2015'])
for year, df in atlas_dict.items():
    new_cols = set(df)

    print("mising in", year, list(og_cols-new_cols))
    print("mising in 2010" , list(new_cols-og_cols))
    print()

mising in 2015 []
mising in 2010 []

mising in 2019 ['POP2010']
mising in 2010 ['Pop2010']



In [80]:
#rename and concat togethor
atlas_dict['2015'] = atlas_dict['2015'].rename(columns= {'POP2010':'Pop2010'})

#combine them together
atlas = pd.concat(atlas_dict).reset_index()


#rename the year column and drop old index
atlas = atlas.rename(columns={'level_0':'Survey_Year'})
del atlas['level_1']



In [81]:
#start converting to long form
atlas = atlas.melt(id_vars= ['Survey_Year', 'CensusTract', 'State','County'])

In [82]:
atlas.head ()

,Survey_Year,CensusTract,State,County,variable,value
0,2015,01001020100,Alabama,Autauga,Urban,1
1,2015,01001020200,Alabama,Autauga,Urban,1
2,2015,01001020300,Alabama,Autauga,Urban,1
3,2015,01001020400,Alabama,Autauga,Urban,1
4,2015,01001020500,Alabama,Autauga,Urban,1


In [83]:
# add a key of what the variables are
atlas_key = pd.read_csv('data/raw/food_atlas_col_key.csv')


#and merge it with atlas
atlas = atlas.merge(atlas_key, left_on='variable', right_on='Column')

del atlas['Column']

In [84]:
#save
atlas.to_csv('data/clean/food_atlas.csv', index=False)

# Join Atlas and CDC


In [3]:
cdc= pd.read_csv('data/clean/cdc_500_cities.csv', dtype='str')
atlas= pd.read_csv('data/clean/food_atlas.csv', dtype='str')
print(cdc.columns)
print(atlas.columns)

Index(['Release_Year', 'Survey_Year', 'StateAbbr', 'StateDesc', 'CityName',
       'GeographicLevel', 'DataSource', 'Category', 'UniqueID', 'Measure',
       'Data_Value_Unit', 'DataValueTypeID', 'Data_Value_Type', 'Data_Value',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Low_Confidence_Limit', 'High_Confidence_Limit', 'Population2010',
       'GeoLocation', 'CategoryID', 'MeasureId', 'CityFIPS', 'TractFIPS',
       'Short_Question_Text'],
      dtype='object')
Index(['Survey_Year', 'CensusTract', 'State', 'County', 'variable', 'value',
       'Data_Value_Unit', 'Description'],
      dtype='object')


In [4]:
#rename atlas cols to fit
atlas = atlas.rename(columns={'CensusTract':'TractFIPS',
                                'State':'StateDesc', 
                                'County':'CityName',
                                'variable': 'MeasureId',
                                'value': 'Data_Value'
                                })

#change vars for memory
#to_cat = ['Survey_Year','StateDesc','CityName','MeasureId','Data_Value_Unit','Description']
#atlas[to_cat] = atlas[to_cat].astype('category')

combo = pd.concat([atlas, cdc], keys =['atlas','cdc']).reset_index()





In [5]:
combo['level_0'].unique()


array(['atlas', 'cdc'], dtype=object)

In [6]:
#get rid of extra cols
cols_keep =[
    'level_0',
    'Survey_Year',
    'TractFIPS',
    'StateDesc', 
    'CityName',
    'MeasureId',
    'Data_Value',
    'Low_Confidence_Limit',
    'High_Confidence_Limit',
    'Data_Value_Type',
    'Data_Value_Unit'
]

combo = combo[cols_keep]

In [7]:
#delet tracts that are not represented in the 500 city data

#get list of ids in cdc data
cdc_tracts = cdc['TractFIPS'].unique()


combo = combo[combo['TractFIPS'].isin(cdc_tracts)]

In [8]:
combo = combo.rename(columns={'level_0':'Data_Source'})

In [9]:
#exaple subsection looking at a tract in Mobile Alabama
combo[ combo['TractFIPS']=='01073000100'].pivot(columns='MeasureId', values='Data_Value', index='Survey_Year')

MeasureId,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CHD,CHECKUP,CHOLSCREEN,...,lasnaphalf,lasnaphalfshare,lawhite1,lawhite10,lawhite10share,lawhite1share,lawhite20,lawhite20share,lawhitehalf,lawhitehalfshare
Survey_Year,,,,,,,,,,,,,,,,,,,,,
2013,NaN,NaN,NaN,46.8,80.3,NaN,NaN,NaN,NaN,72.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,27.6,34.0,10.3,NaN,NaN,5.5,12.8,7.8,76.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,23.9,32.5,10.1,46.2,80.0,5.2,12.7,8.0,76.3,72.3,...,231.706749067525,0.205413784634331,119.121656463714,0,0,0.039158992920353,0,0,500.502902490319,0.164530868668744
2017,24.4,31.2,10.3,47.7,77.8,5.3,13.4,7.8,74.8,78.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,311.511230919627,27.616243875853403,119.121656463714,NaN,NaN,3.91589929203532,NaN,NaN,500.50290249032,16.4530868668744


In [10]:
#fill in data type so we can agg
combo['Data_Value_Type'] = combo['Data_Value_Type'].fillna('unsure')

In [54]:
#some of the tracs got truncated at some point add the 0 to the front if so
def add_0(x):
    if len(x)==10:
        return "0"+x
    else:
        return x
combo['TractFIPS'] = combo['TractFIPS'].apply(add_0)

In [55]:
combo.to_csv('data/clean/atlas_cdc_combined.csv', index=False)

# Converting to county level

In [12]:
#first we need to combine our old data into county level
#for tractFIPS
#State Code: The first two digits represent the state in which the census tract is located. 
#County Code: The next three digits specify the county within that state. 
#Tract Code: The final six digits identify the specific census tract within the county. 
#Full ID: The complete 11-digit code is a concatenation of these components, for example, state_code + county_code + tract_code. 
# for seq = '01073000100'
# The state + county is  seq[0:5]


In [13]:
combo[combo['Data_Source']=='cdc']

,Data_Source,Survey_Year,TractFIPS,StateDesc,CityName,MeasureId,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,Data_Value_Type,Data_Value_Unit
20936880,cdc,2014,01073013002,Alabama,Birmingham,TEETHLOST,31.9,20.4,43.9,Crude prevalence,%
20936881,cdc,2014,01125011402,Alabama,Tuscaloosa,TEETHLOST,24.2,18.1,31.2,Crude prevalence,%
20936882,cdc,2014,01073004901,Alabama,Birmingham,TEETHLOST,11.2,8.0,15.6,Crude prevalence,%
20936883,cdc,2014,01073000700,Alabama,Birmingham,TEETHLOST,45.8,35.2,56.1,Crude prevalence,%
20936884,cdc,2014,01097001501,Alabama,Mobile,TEETHLOST,43.9,33.1,54.0,Crude prevalence,%
...,...,...,...,...,...,...,...,...,...,...,...
23509047,cdc,2017,55025000202,Wisconsin,Madison,COPD,3.8,3.2,4.6,Crude prevalence,%
23509048,cdc,2017,53053061900,Washington,Tacoma,CHOLSCREEN,74.2,73.1,75.3,Crude prevalence,%
23509049,cdc,2017,53011041331,Washington,Vancouver,HIGHCHOL,28.1,27.3,29.0,Crude prevalence,%
23509050,cdc,2017,55087010200,Wisconsin,Appleton,BINGE,27.0,26.0,28.0,Crude prevalence,%


In [14]:
combo['Data_Value_Unit'].unique()

array(['%', 'T'], dtype=object)

In [56]:
#add a coumn for county code
combo['CountyFIPS'] = combo['TractFIPS'].str[0:5]

#split into datatypes with totats 'T' and percent %
percent_df = combo[combo['Data_Value_Unit']=='%']
sum_df = combo[combo['Data_Value_Unit']=='T']



In [57]:
#and gett aggragate vals from each

cols_agg ={
    'Data_Source':'first',
    'StateDesc': 'first', 
    'CityName': 'first',
    'Data_Value_Unit': 'first',
    'Data_Value': 'sum',
    'Low_Confidence_Limit': 'sum',
    'High_Confidence_Limit': 'sum',
    'Data_Value_Unit': 'first'
}

sum_df['Data_Value'] = sum_df['Data_Value'].astype('float32')
sum_df = sum_df.groupby(['CountyFIPS', 'Survey_Year', 'MeasureId', 'Data_Value_Type']).agg(cols_agg).reset_index()


C:\Users\alecs\AppData\Local\Temp\ipykernel_48352\848387131.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Data_Value'] = sum_df['Data_Value'].astype('float32')


In [58]:
sum_df.head()

,CountyFIPS,Survey_Year,MeasureId,Data_Value_Type,Data_Source,StateDesc,CityName,Data_Value_Unit,Data_Value,Low_Confidence_Limit,High_Confidence_Limit
0,01073,2015,LALOWI05_10,unsure,atlas,Alabama,Jefferson,T,120045.960938,0,0
1,01073,2015,LALOWI1_10,unsure,atlas,Alabama,Jefferson,T,63290.210938,0,0
2,01073,2015,LALOWI1_20,unsure,atlas,Alabama,Jefferson,T,62352.031250,0,0
3,01073,2015,LAPOP05_10,unsure,atlas,Alabama,Jefferson,T,308489.531250,0,0
4,01073,2015,LAPOP1_10,unsure,atlas,Alabama,Jefferson,T,166604.078125,0,0


In [77]:
# get the population at each census
pop_counts_county = sum_df[sum_df['MeasureId']=='Pop2010'].drop_duplicates(subset=['CountyFIPS'])
pop_counts_county = pop_counts_county[['CountyFIPS','Data_Value']].set_index('CountyFIPS')
pop_counts_county['Data_Value'] = pop_counts_county['Data_Value'].astype('float32')

#and for tract
pop_counts_tract = combo[combo['MeasureId']=='Pop2010'].drop_duplicates(subset=['TractFIPS','Data_Value'])
pop_counts_tract = pop_counts_tract[['TractFIPS','Data_Value']].set_index('TractFIPS')
pop_counts_tract['Data_Value'] = pop_counts_tract['Data_Value'].astype('float32')

In [ ]:
#first multiply by pop in county
def weighting(x):
    county_code = x[0:5]
    return pop_counts_tract.loc[x]/pop_counts_county.loc[county_code]


percent_df['tract_pop']=percent_df['TractFIPS'].apply(lambda x: pop_counts_tract.loc[x] )
percent_df['county_pop']=percent_df['TractFIPS'].apply(lambda x: pop_counts_county.loc[x[0:5]] )
 

C:\Users\alecs\AppData\Local\Temp\ipykernel_48352\1104654289.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  percent_df['county_pop']=percent_df['TractFIPS'].apply(lambda x: pop_counts_county.loc[x[0:5]] )


In [80]:
percent_df['weights'] = percent_df['tract_pop']/percent_df['county_pop']

C:\Users\alecs\AppData\Local\Temp\ipykernel_48352\2863664607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  percent_df['weights'] = percent_df['tract_pop']/percent_df['county_pop']


In [ ]:
percent_df2['we']

,CountyFIPS,Survey_Year,MeasureId,Data_Value_Type,Data_Source,StateDesc,CityName,Data_Value_Unit
0,01073,2013,BPHIGH,Crude prevalence,cdc,Alabama,Birmingham,%
1,01073,2013,BPMED,Crude prevalence,cdc,Alabama,Birmingham,%
2,01073,2013,CHOLSCREEN,Crude prevalence,cdc,Alabama,Birmingham,%
3,01073,2013,HIGHCHOL,Crude prevalence,cdc,Alabama,Hoover,%
4,01073,2014,ACCESS2,Crude prevalence,cdc,Alabama,Birmingham,%
...,...,...,...,...,...,...,...,...
78601,56021,2019,lasnaphalfshare,unsure,atlas,Wyoming,Laramie County,%
78602,56021,2019,lawhite10share,unsure,atlas,Wyoming,Laramie County,%
78603,56021,2019,lawhite1share,unsure,atlas,Wyoming,Laramie County,%
78604,56021,2019,lawhite20share,unsure,atlas,Wyoming,Laramie County,%


In [95]:
#now do the same for the precent
#we have to do a weighted avg 
def weighted_average(df):
    #data_cols=['Data_Value','Low_Confidence_Limit','High_Confidence_Limit']
    return (df['weights'] * df['Data_Value']).sum() 


cols_agg ={
    'Data_Source':'first',
    'StateDesc': 'first', 
    'CityName': 'first',
    'Data_Value_Unit': 'first',
    'Data_Value_Unit': 'first'
}
percent_df2=percent_df
percent_df2['Data_Value'] = percent_df2['Data_Value'].astype('float32')
percent_df2 = percent_df2.groupby(['CountyFIPS', 'Survey_Year', 'MeasureId', 'Data_Value_Type']).agg(cols_agg).reset_index()



data_cols = percent_df.groupby(['CountyFIPS', 'Survey_Year', 'MeasureId', 'Data_Value_Type']).apply(weighted_average)
data_cols

C:\Users\alecs\AppData\Local\Temp\ipykernel_48352\2781029478.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  percent_df2['Data_Value'] = percent_df2['Data_Value'].astype('float32')
C:\Users\alecs\AppData\Local\Temp\ipykernel_48352\2781029478.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_cols = percent_df.groupby(['CountyFIPS', 'Survey_Year', 'MeasureId', 'Data_Value_Type']).apply(weighted_average)


CountyFIPS  Survey_Year  MeasureId         Data_Value_Type 
01073       2013         BPHIGH            Crude prevalence    39.382385
                         BPMED             Crude prevalence    75.504128
                         CHOLSCREEN        Crude prevalence    71.890869
                         HIGHCHOL          Crude prevalence    36.702557
            2014         ACCESS2           Crude prevalence    17.195215
                                                                 ...    
56021       2019         lasnaphalfshare   unsure               5.482672
                         lawhite10share    unsure               4.776852
                         lawhite1share     unsure              37.410759
                         lawhite20share    unsure               0.257093
                         lawhitehalfshare  unsure              70.659744
Length: 78606, dtype: float32

In [ ]:
#combine data and info cols
percent_df2['Data_Value']=data_cols.reset_index()[0]
percent_df2.head()

,CountyFIPS,Survey_Year,MeasureId,Data_Value_Type,Data_Source,StateDesc,CityName,Data_Value_Unit,Data_Value
0,01073,2013,BPHIGH,Crude prevalence,cdc,Alabama,Birmingham,%,39.382385
1,01073,2013,BPMED,Crude prevalence,cdc,Alabama,Birmingham,%,75.504128
2,01073,2013,CHOLSCREEN,Crude prevalence,cdc,Alabama,Birmingham,%,71.890869
3,01073,2013,HIGHCHOL,Crude prevalence,cdc,Alabama,Hoover,%,36.702557
4,01073,2014,ACCESS2,Crude prevalence,cdc,Alabama,Birmingham,%,17.195215


In [113]:
#and concat back together
county_combo = pd.concat([percent_df2, sum_df]).reset_index()


In [115]:
county_combo.to_csv('data/clean/atlas_cdc_combined_county_level.csv', index=False)

# Feeding America

In [43]:
file_list=[
    'data/raw/feeding_america/MMG2015_2013Data_ToShare.xlsx',
    'data/raw/feeding_america/MMG2016_2014Data_ToShare.xlsx',
    'data/raw/feeding_america/MMG2017_2015Data_ToShare.xlsx',
    'data/raw/feeding_america/MMG2018_2016Data_ToShare.xlsx',
    'data/raw/feeding_america/MMG2019_2017Data_ToShare.xlsx'
]

feeding_dict = {} #each year as a string will be key
for file in file_list:
    #get the release year
    release_year = re.search(r'([0-9]{4})Data', file).group(1)

    #read in the data
    df = pd.read_excel(file, sheet_name = 0, dtype='str')
    feeding_dict[release_year] = df


In [44]:
#check all cols are same
og_cols = set(feeding_dict['2014'])
for year, df in feeding_dict.items():
    new_cols = set(df)

    print("mising in", year, list(og_cols-new_cols))
    print("missing in 2013" , list(new_cols-og_cols))
    print()

mising in 2013 ['2014 Weighted Annual Food Budget Shortfall', '% food insecure children in HH w/ HH incomes above 185 FPL in 2014', '% food insecure children in HH w/ HH incomes below 185 FPL in 2014', '2014 Child food insecurity rate', '# of Food Insecure Persons in 2014', 'County, State', '# of Food Insecure Children in 2014', '2014 Cost Per Meal', '2014 Food Insecurity Rate']
missing in 2013 ['# of Food Insecure Children in 2013', '# of Food Insecure Persons in 2013 ', '2013 Food Insecurity Rate', 'State Name', '% food insecure Children in HH w/HH Incomes Above 185 FPL in 2013', '2013 Child Food Insecurity Rate', '2013 Cost Per Meal', '2013 Weighted Annual Food Budget Shortfall', '% food insecure Children in HH w/HH Incomes Below 185 FPL in 2013']

mising in 2014 []
missing in 2013 []

mising in 2015 ['2014 Weighted Annual Food Budget Shortfall', '% food insecure children in HH w/ HH incomes above 185 FPL in 2014', '% food insecure children in HH w/ HH incomes below 185 FPL in 2014'

In [45]:
#so the diff in cols is the year


In [46]:
import re

def remove_years(strings):
    """
    Removes year (four consecutive digits) from each string in a list.

    Args:
        strings (list of str): List of strings to process.

    Returns:
        list of str: Strings with years removed.
    """
    return [re.sub(r'( in )?\d{4}', '', s).strip() for s in strings]

# Example usage
example_list = ['data in 2023', 'report 2019', 'summary 2020']
cleaned_list = remove_years(example_list)
print(remove_years(feeding_dict['2015'].columns)) 
print(feeding_dict['2015'].columns)
 # Output: ['data_', 'report_', 'summary_']

['FIPS', 'State', 'County, State', 'Food Insecurity Rate', '# of Food Insecure Persons', 'Low Threshold in state', 'Low Threshold Type', 'High Threshold in state', 'High Threshold Type', '% FI ≤ Low Threshold', '% FI Btwn Thresholds', '% FI > High Threshold', 'Child food insecurity rate', '# of Food Insecure Children', '% food insecure children in HH w/ HH incomes below 185 FPL', '% food insecure children in HH w/ HH incomes above 185 FPL', 'Cost Per Meal', 'Weighted Annual Food Budget Shortfall']
Index(['FIPS', 'State', 'County, State', '2015 Food Insecurity Rate',
       '# of Food Insecure Persons in 2015', 'Low Threshold in state',
       'Low Threshold Type', 'High Threshold in state', 'High Threshold Type',
       '% FI ≤ Low Threshold', '% FI Btwn Thresholds', '% FI > High Threshold',
       '2015 Child food insecurity rate',
       '# of Food Insecure Children in 2015',
       '% food insecure children in HH w/ HH incomes below 185 FPL in 2015',
       '% food insecure children

In [47]:
for key in feeding_dict:
    old_cols = feeding_dict[key].columns
    feeding_dict[key].columns = remove_years(old_cols)

#check all cols are same
og_cols = set(feeding_dict['2014'])
for year, df in feeding_dict.items():
    new_cols = set(df)

    print("mising in", year, list(og_cols-new_cols))
    print("missing in 2013" , list(new_cols-og_cols))
    print()

mising in 2013 ['County, State', '% food insecure children in HH w/ HH incomes above 185 FPL', '% food insecure children in HH w/ HH incomes below 185 FPL', 'Child food insecurity rate']
missing in 2013 ['State Name', 'Child Food Insecurity Rate', '% food insecure Children in HH w/HH Incomes Above 185 FPL', '% food insecure Children in HH w/HH Incomes Below 185 FPL']

mising in 2014 []
missing in 2013 []

mising in 2015 []
missing in 2013 []

mising in 2016 []
missing in 2013 []

mising in 2017 []
missing in 2013 []



In [48]:
#okay im just going to delet 2013
del feeding_dict['2013']

In [49]:
#combine them together
feeding = pd.concat(feeding_dict).reset_index()

#rename the year column and drop old index
feeding = feeding.rename(columns={'level_0':'Survey_Year'})
del feeding['level_1']

feeding.head()

,Survey_Year,FIPS,State,"County, State",Food Insecurity Rate,# of Food Insecure Persons,Low Threshold in state,Low Threshold Type,High Threshold in state,High Threshold Type,% FI ≤ Low Threshold,% FI Btwn Thresholds,% FI > High Threshold,Child food insecurity rate,# of Food Insecure Children,% food insecure children in HH w/ HH incomes below 185 FPL,% food insecure children in HH w/ HH incomes above 185 FPL,Cost Per Meal,Weighted Annual Food Budget Shortfall
0,2014,1001,AL,"Autauga County, Alabama",0.144,7930,1.3,SNAP,1.85,Other Nutrition Program,0.451,0.193,0.356,0.224,3190,0.72,0.28,3.16,4425000
1,2014,1003,AL,"Baldwin County, Alabama",0.137,26200,1.3,SNAP,1.85,Other Nutrition Program,0.492,0.154,0.354,0.238,10270,0.73,0.27,3.38,15630000
2,2014,1005,AL,"Barbour County, Alabama",0.233,6330,1.3,SNAP,1.85,Other Nutrition Program,0.501,0.212,0.287,0.3,1760,0.74,0.26,3.02,3377000
3,2014,1007,AL,"Bibb County, Alabama",0.164,3730,1.3,SNAP,1.85,Other Nutrition Program,0.566,0.24,0.194,0.266,1310,0.97,0.03,2.91,1914000
4,2014,1009,AL,"Blount County, Alabama",0.122,7030,1.3,SNAP,1.85,Other Nutrition Program,0.628,0.141,0.231,0.255,3540,0.77,0.23,2.91,3613000


In [51]:
#Some of the FIPS are too short again
def add_0(x):
    if len(x)==4:
        return "0"+x
    else:
        return x
    
feeding['FIPS']= feeding['FIPS'].apply(add_0)



In [53]:
feeding.columns

Index(['Survey_Year', 'FIPS', 'State', 'County, State', 'Food Insecurity Rate',
       '# of Food Insecure Persons', 'Low Threshold in state',
       'Low Threshold Type', 'High Threshold in state', 'High Threshold Type',
       '% FI ≤ Low Threshold', '% FI Btwn Thresholds', '% FI > High Threshold',
       'Child food insecurity rate', '# of Food Insecure Children',
       '% food insecure children in HH w/ HH incomes below 185 FPL',
       '% food insecure children in HH w/ HH incomes above 185 FPL',
       'Cost Per Meal', 'Weighted Annual Food Budget Shortfall'],
      dtype='object')

In [55]:
feeding_long= feeding.melt(
    id_vars=['Survey_Year', 'FIPS', 'State', 'County, State'],
    var_name='MeasureId',
    value_name='Data_Value'
)
feeding_long.head()

,Survey_Year,FIPS,State,"County, State",MeasureId,Data_Value
0,2014,01001,AL,"Autauga County, Alabama",Food Insecurity Rate,0.144
1,2014,01003,AL,"Baldwin County, Alabama",Food Insecurity Rate,0.137
2,2014,01005,AL,"Barbour County, Alabama",Food Insecurity Rate,0.233
3,2014,01007,AL,"Bibb County, Alabama",Food Insecurity Rate,0.164
4,2014,01009,AL,"Blount County, Alabama",Food Insecurity Rate,0.122


In [59]:
#split the county and state

feeding_long[['CityName','StateDesc']] = feeding_long['County, State'].str.split(pat=',', expand=True)

In [62]:
del feeding_long['State' ]
del feeding_long['County, State' ]


In [64]:
feeding_long = feeding_long.rename(columns={'FIPS':'CountyFIPS' })

In [65]:
feeding_long.head()

,Survey_Year,CountyFIPS,MeasureId,Data_Value,CityName,StateDesc
0,2014,01001,Food Insecurity Rate,0.144,Autauga County,Alabama
1,2014,01003,Food Insecurity Rate,0.137,Baldwin County,Alabama
2,2014,01005,Food Insecurity Rate,0.233,Barbour County,Alabama
3,2014,01007,Food Insecurity Rate,0.164,Bibb County,Alabama
4,2014,01009,Food Insecurity Rate,0.122,Blount County,Alabama


In [66]:
feeding.to_csv('data/clean/feeding_america_wide.csv', index=False)

# Combining all togethor


In [67]:
all_county = pd.read_csv('data/clean/atlas_cdc_combined_county_level.csv', dtype='str')


In [69]:
all_county = pd.concat([all_county, feeding_long]).reset_index()
all_county

,level_0,index,CountyFIPS,Survey_Year,MeasureId,Data_Value_Type,Data_Source,StateDesc,CityName,Data_Value_Unit,Data_Value,Low_Confidence_Limit,High_Confidence_Limit
0,0,0,01073,2013,BPHIGH,Crude prevalence,cdc,Alabama,Birmingham,%,39.382385,NaN,NaN
1,1,1,01073,2013,BPMED,Crude prevalence,cdc,Alabama,Birmingham,%,75.50413,NaN,NaN
2,2,2,01073,2013,CHOLSCREEN,Crude prevalence,cdc,Alabama,Birmingham,%,71.89087,NaN,NaN
3,3,3,01073,2013,HIGHCHOL,Crude prevalence,cdc,Alabama,Hoover,%,36.702557,NaN,NaN
4,4,4,01073,2014,ACCESS2,Crude prevalence,cdc,Alabama,Birmingham,%,17.195215,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
317293,188515,NaN,56037,2017,Weighted Annual Food Budget Shortfall,NaN,NaN,Wyoming,Sweetwater County,NaN,2542000,NaN,NaN
317294,188516,NaN,56039,2017,Weighted Annual Food Budget Shortfall,NaN,NaN,Wyoming,Teton County,NaN,1592000,NaN,NaN
317295,188517,NaN,56041,2017,Weighted Annual Food Budget Shortfall,NaN,NaN,Wyoming,Uinta County,NaN,1340000,NaN,NaN
317296,188518,NaN,56043,2017,Weighted Annual Food Budget Shortfall,NaN,NaN,Wyoming,Washakie County,NaN,497000,NaN,NaN


In [70]:
# fill in missing data source
all_county['Data_Source'] = all_county['Data_Source'].fillna('Feeding_America')

In [72]:
#delet unused indexing cols
del all_county['level_0']
del all_county['index']


In [73]:
#and save 
feeding.to_csv('data/clean/all_datasets_county_level.csv', index=False)